In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
df = pd.read_csv('database.csv')

In [87]:
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [88]:
df.shape

(23412, 21)

In [89]:
df = df[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]

In [90]:
df.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,01/02/1965,13:44:18,19.246,145.616,131.6,6.0
1,01/04/1965,11:29:49,1.863,127.352,80.0,5.8
2,01/05/1965,18:05:58,-20.579,-173.972,20.0,6.2
3,01/08/1965,18:49:43,-59.076,-23.557,15.0,5.8
4,01/09/1965,13:32:50,11.938,126.427,15.0,5.8


In [91]:
df.isnull().sum()

,0
Date,0
Time,0
Latitude,0
Longitude,0
Depth,0
Magnitude,0


In [92]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%m/%d/%Y')

In [93]:
minutes = []
hours = []
seconds = []

for t in df.Time:
    t  = t.split(":")
    minutes.append(t[1])
    hours.append(t[0])
    seconds.append(t[2])

In [94]:
df['minutes'] = minutes
df['hours'] = hours
df['seconds'] = seconds

df['minutes'] = pd.to_numeric(df['minutes'], errors='coerce')
df['hours'] = pd.to_numeric(df['hours'], errors='coerce')
df['seconds'] = pd.to_numeric(df['seconds'], errors='coerce')

In [95]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [96]:
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [97]:
df.head()

,Latitude,Longitude,Depth,Magnitude,minutes,hours,seconds,year,month,day
0,19.246,145.616,131.6,6.0,44,13.0,18.0,1965.0,1.0,2.0
1,1.863,127.352,80.0,5.8,29,11.0,49.0,1965.0,1.0,4.0
2,-20.579,-173.972,20.0,6.2,5,18.0,58.0,1965.0,1.0,5.0
3,-59.076,-23.557,15.0,5.8,49,18.0,43.0,1965.0,1.0,8.0
4,11.938,126.427,15.0,5.8,32,13.0,50.0,1965.0,1.0,9.0


In [98]:
df.dropna(inplace=True)

In [99]:
df.isnull().sum()

,0
Latitude,0
Longitude,0
Depth,0
Magnitude,0
minutes,0
hours,0
seconds,0
year,0
month,0
day,0


In [100]:
X = df[['Latitude', 'Longitude', 'minutes', 'hours', 'seconds', 'year', 'month', 'day']]
y = df[['Depth', 'Magnitude']]

In [101]:
X = X.values
y = y.values

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [103]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [104]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16386, 8) (7023, 8) (16386, 2) (7023, 2)


In [105]:
## creating a neural Network model

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(8,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [106]:
model.compile(optimizer='adam', loss='squared_hinge', metrics=['accuracy'])

In [107]:
history = model.fit(X_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9816 - loss: 0.0036 - val_accuracy: 0.9825 - val_loss: 0.0033
Epoch 2/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9793 - loss: 0.0045 - val_accuracy: 0.9623 - val_loss: 0.0033
Epoch 3/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9792 - loss: 0.0037 - val_accuracy: 0.9737 - val_loss: 0.0033
Epoch 4/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9744 - loss: 0.0041 - val_accuracy: 0.9616 - val_loss: 0.0033
Epoch 5/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9655 - loss: 0.0039 - val_accuracy: 0.9423 - val_loss: 0.0033
Epoch 6/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9208 - loss: 0.0032 - val_accuracy: 0.9571 - val_loss: 0.0033
Epoch 7/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9333 - loss: 0.0042 - val_accuracy: 0.9446 - val_loss: 0.0033
Epoch 8/20
257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8639 - loss: 0.0041 - val_accuracy: 0.

In [108]:
import joblib

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [109]:
from tensorflow.keras.models import save_model

save_model(model, "earthquake_prediction_model.keras")